In [1]:
from time import perf_counter
from pyci.configint.rcisd import CISD
from threadpoolctl import threadpool_limits
import os
os.environ['PSI_SCRATCH']='/data1/vijay/jcp_helium_paper/psi_scratch/'
import psi4
import pyci
import numpy as np
import sys

In [26]:
# Lets intiate AOint() a subclass of psi4utils()
molfile = '../.xyz/Be.xyz'
ncore = 22
with threadpool_limits(limits=ncore, user_api='blas'):
    mol = pyci.utils.molecule('aug-cc-pvqz', molfile=molfile, wd='./', ncore=ncore, psi4mem='210 Gb', numpymem=210,
                              custom_basis=False, store_wfn=False, properties=['dipoles'], psi4options={'puream': True})
with threadpool_limits(limits=ncore, user_api='blas'):
    cisd = CISD(mol, 
                options={'singles': True,
                         'full_cis': True,
                         'doubles': True,
                         'doubles_iiaa': True,
                         'doubles_iiab': True,
                         'doubles_ijaa': True,
                         'doubles_ijab_A': True,
                         'doubles_ijab_B': True}, ncore=ncore)
    ndim = sum(cisd.num_csfs)
    print(cisd.num_csfs, ndim)
    cisd.save_hcisd()
print('Diagonalizing HCISD matrix.....\n')
HCISD = np.load('cimat.npz')['HCISD']
cisd_pyci = cisd.energy(HCISD)
if cisd.options['doubles']:
    cisd_psi4 = psi4.energy('CISD')
else:
    cisd_psi4 = psi4.energy('scf')
print("pyci CISD E0: {e:16.16f}".format(e=cisd_pyci))
print("psi4 CISD E0: {e:16.16f}".format(e=cisd_psi4))
print("dE : {dE:1.2E}\n".format(dE=abs(cisd_psi4 - cisd_pyci)))


COMPUTING AO ELECTRON REPULSION INTEGRALS

Size of the ERI tensor will be 0.33 Gb

COMPUTING AO ONE ELECTRON INTEGRALS

Ground state SCF Energy : -14.57296919 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -14.57296919 

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 80  
Molecular orbitals: 80  
Spin Orbitals     : 160  
Alpha Orbitals    : 2  
Beta Orbitals     : 2  
Occupied Orbitals : 2  
Virtual Orbitals  : 78  

[1, 156, 156, 6006, 78, 3003, 3003] 12403
Computing CISD Hamiltonian matrix with 22 cores 

Diagonalizing HCISD matrix.....

pyci CISD E0: -14.6381119078707052
psi4 CISD E0: -14.6381119078707371
dE : 3.20E-14



In [27]:
np.allclose(HCISD, HCISD.T)

True

In [28]:
num_csfs = cisd.num_csfs
idx = [sum(num_csfs[:i]) for i in range(1,8)]

In [29]:
idx

[1, 157, 313, 6319, 6397, 9400, 12403]

In [30]:
i = 0
block = HCISD[idx[i]:idx[i+1], idx[i]:idx[i+1]]
cum_block = HCISD[:idx[i+1], :idx[i+1]]

In [31]:
print(block.shape)
print('block', np.allclose(block, block.T))
print(cum_block.shape)
print('cum_block', np.allclose(block, block.T))


(156, 156)
block True
(157, 157)
cum_block True
